In [1]:
#-*- coding: utf-8 -*-
#import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import json
import numpy as np

# 1. 박스스코어에서
http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=0&gameId=20160401LTWO0&gyear=2016

In [8]:
### 미완성 ###

LeagueID = ['1']
SeriesID = ['0', '1', '4']
#YearList = ['2010', '2011', '2012', '2013', '2014', '2015', '2016']
YearList = ['2016']
MonthList= ['04', '05', '06', '07', '08', '09', '10', '11', '12']
DayList = ['01', '02', '03', '04', '05']
Team = 'HH'
#TeamList = [' ']

#SeasonName: id=cphContainer_cphContents_lblTitle
#SeasonSelect: id=cphContainer_cphContents_ddlGame
#TeamSelect: class=scheduleTab, data-id=HH
#ScheduleBoard: class=scheduleBoard
#                tlable->tbody->td(time, play, relay)

#schedule: http://www.koreabaseball.com/Schedule/GameList/General.aspx
#BoxScore: http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=0&gameId=20160401HHLG0&gyear=2016
#          seriesid(0=정규, 1=시범, 4=포스트), gameid(yyyy/mm/dd/t1/t2/0), gyear(yyyy)


#driver = webdriver.Chrome("/home/jks207/chromedriver")
SiteURL = ['http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?']



for league in LeagueID:
    for series in SeriesID:
        for year in YearList:
            for month in MonthList:
                for day in DayList:
                    SiteURL[0] = SiteURL[0] + 'leagueId='+ league + '&' +'seriesId=' + series +'&'+ 'gameId='+ year + month + day
                    print SiteURL

['http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=0&gameId=20160401']
['http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=0&gameId=20160401leagueId=1&seriesId=0&gameId=20160402']
['http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=0&gameId=20160401leagueId=1&seriesId=0&gameId=20160402leagueId=1&seriesId=0&gameId=20160403']
['http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=0&gameId=20160401leagueId=1&seriesId=0&gameId=20160402leagueId=1&seriesId=0&gameId=20160403leagueId=1&seriesId=0&gameId=20160404']
['http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=0&gameId=20160401leagueId=1&seriesId=0&gameId=20160402leagueId=1&seriesId=0&gameId=20160403leagueId=1&seriesId=0&gameId=20160404leagueId=1&seriesId=0&gameId=20160405']
['http://www.koreabaseball.com/Schedule/Game/BoxScore.aspx?leagueId=1&seriesId=0&gameId=20160401leagueId=1&seriesId=0&gameId=20160402le

# 2. 경기일정/결과에서
http://www.koreabaseball.com/Schedule/GameList/General.aspx

In [2]:
### 데이터 파싱 ###

#File = open("KBO0212.txt", 'a')
#if(File != open):
#    print 'file error'

YearList = ['2010', '2011', '2012', '2013', '2014', '2015', '2016']
MonthList= ['03', '04', '05', '06', '07', '08', '09', '10']
result = []

#YearList=['2011']
#MonthList=['04']
Team = 'HH'

driver = webdriver.Chrome("/home/jks207/chromedriver")
SiteURL = ['http://www.koreabaseball.com/Schedule/Schedule.aspx']
for url in SiteURL:
    driver.get(url)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    i=0
    for year in YearList:
        selectYear = Select(driver.find_element_by_id("ddlYear"))
        selectYear.select_by_value(year)
        time.sleep(2)
        if year=='2010':
            MonthList2 = MonthList[:-1]
        elif year=='2014' or year=='2015':
            MonthList2 = MonthList
        else:
            MonthList2 = MonthList[1:]
        for month in MonthList2:
            selectMonth = Select(driver.find_element_by_id("ddlMonth"))
            selectMonth.select_by_value(month)
            time.sleep(2)
            
            driver.find_element_by_xpath("//img[contains(@src,'resources/images/emblem/initial_HH.png')]/parent::a").click()
            time.sleep(2)
            
            html_r = driver.page_source           
            soup_r = BeautifulSoup(html_r, "lxml") 
            
            #GameTitle = soup_r.find('span', attrs={'id':"cphContainer_cphContents_lblTitle"})
            
            table      = soup_r.find('table', attrs={'class':'tbl'})
            table_body = table.find('tbody')                             
            rows       = table_body.find_all('tr')    
            
            print year+ month
            #+ GameTitle.find(text=True)
            for row in rows:
                Day = row.find('td', attrs={'class':"day"})
                Time = row.find('td', attrs={'class':"time"})
                Play = row.find('td', attrs={'class':"play"}).find_all('span')
                relay = row.find('td', attrs={'class':"relay"})
                
                result.append([])
                result[i].append(Day.find(text=True).encode('utf-8'))
                result[i].append(Time.find(text=True).encode('utf-8'))
                result[i].append([])
                for play in Play:
                    result[i][2].append(play.find(text=True).encode('utf-8'))
                if((row.find('td').find(text=True).encode('utf-8')=='우천취소') or (row.find('td').find(text=True).encode('utf-8')=='기타')):
                    result[i].append("우천취소")
                else:
                    result[i].append("?")
                i = i+1

#File.close()
print 'finish'

201003
201004
201005
201006
201007
201008
201009
201104
201105
201106
201107
201108
201109
201110
201204
201205
201206
201207
201208
201209
201210
201304
201305
201306
201307
201308
201309
201310
201403
201404
201405
201406
201407
201408
201409
201410
201503
201504
201505
201506
201507
201508
201509
201510
201604
201605
201606
201607
201608
201609
201610
finish


In [3]:
### 한글 팀명 -> 영어 팀명 ###

j, k = 0, -1
for i in range(len(result)):
    if(result[i][2][j]=='삼성'):
        result[i][2][j] = 'SAM'
    elif(result[i][2][j]=='한화'):
        result[i][2][j] = 'HAN'
    elif(result[i][2][j]=='두산'):
        result[i][2][j] = 'DUS'
    elif(result[i][2][j]=='롯데'):
        
        result[i][2][j] = 'LOT'
    elif(result[i][2][j]=='넥센'):
        result[i][2][j] = 'NEX'
for i in range(len(result)):
    if(result[i][2][k]=='삼성'):
        result[i][2][k] = 'SAM'
    elif(result[i][2][k]=='한화'):
        result[i][2][k] = 'HAN'
    elif(result[i][2][k]=='두산'):
        result[i][2][k] = 'DUS'
    elif(result[i][2][k]=='롯데'):
        result[i][2][k] = 'LOT'
    elif(result[i][2][k]=='넥센'):
        result[i][2][k] = 'NEX'

print result

[['03.18(\xeb\xaa\xa9)', '13:00', ['HAN', 'vs', 'LG'], '?'], ['03.27(\xed\x86\xa0)', '14:00', ['HAN', '2', 'vs', '3', 'SK'], '?'], ['03.28(\xec\x9d\xbc)', '14:00', ['HAN', '3', 'vs', '6', 'SK'], '?'], ['03.30(\xed\x99\x94)', '18:30', ['LOT', '3', 'vs', '13', 'HAN'], '?'], ['03.31(\xec\x88\x98)', '18:30', ['LOT', 'vs', 'HAN'], '?'], ['04.01(\xeb\xaa\xa9)', '18:30', ['LOT', 'vs', 'HAN'], '?'], ['04.02(\xea\xb8\x88)', '18:30', ['SAM', '8', 'vs', '1', 'HAN'], '?'], ['04.03(\xed\x86\xa0)', '17:00', ['SAM', '9', 'vs', '11', 'HAN'], '?'], ['04.04(\xec\x9d\xbc)', '17:00', ['SAM', '2', 'vs', '1', 'HAN'], '?'], ['04.06(\xed\x99\x94)', '18:30', ['HAN', '2', 'vs', '3', 'DUS'], '?'], ['04.07(\xec\x88\x98)', '18:30', ['HAN', '2', 'vs', '8', 'DUS'], '?'], ['04.08(\xeb\xaa\xa9)', '18:30', ['HAN', '4', 'vs', '5', 'DUS'], '?'], ['04.09(\xea\xb8\x88)', '18:30', ['HAN', '15', 'vs', '14', 'LOT'], '?'], ['04.10(\xed\x86\xa0)', '17:00', ['HAN', '9', 'vs', '2', 'LOT'], '?'], ['04.11(\xec\x9d\xbc)', '14:00', [

In [4]:
### vs제거 ###

for i in range(len(result)):
    for j in range(len(result[i][2])):
        if result[i][2][j] == 'vs':
            del result[i][2][j]
            break
            
print result

[['03.18(\xeb\xaa\xa9)', '13:00', ['HAN', 'LG'], '?'], ['03.27(\xed\x86\xa0)', '14:00', ['HAN', '2', '3', 'SK'], '?'], ['03.28(\xec\x9d\xbc)', '14:00', ['HAN', '3', '6', 'SK'], '?'], ['03.30(\xed\x99\x94)', '18:30', ['LOT', '3', '13', 'HAN'], '?'], ['03.31(\xec\x88\x98)', '18:30', ['LOT', 'HAN'], '?'], ['04.01(\xeb\xaa\xa9)', '18:30', ['LOT', 'HAN'], '?'], ['04.02(\xea\xb8\x88)', '18:30', ['SAM', '8', '1', 'HAN'], '?'], ['04.03(\xed\x86\xa0)', '17:00', ['SAM', '9', '11', 'HAN'], '?'], ['04.04(\xec\x9d\xbc)', '17:00', ['SAM', '2', '1', 'HAN'], '?'], ['04.06(\xed\x99\x94)', '18:30', ['HAN', '2', '3', 'DUS'], '?'], ['04.07(\xec\x88\x98)', '18:30', ['HAN', '2', '8', 'DUS'], '?'], ['04.08(\xeb\xaa\xa9)', '18:30', ['HAN', '4', '5', 'DUS'], '?'], ['04.09(\xea\xb8\x88)', '18:30', ['HAN', '15', '14', 'LOT'], '?'], ['04.10(\xed\x86\xa0)', '17:00', ['HAN', '9', '2', 'LOT'], '?'], ['04.11(\xec\x9d\xbc)', '14:00', ['HAN', '9', '10', 'LOT'], '?'], ['04.13(\xed\x99\x94)', '18:30', ['SK', '1', '2', 'H

In [8]:
### 한화의 각 경기 당 점수차(2011.04~2016.10) ###

score = []
for i in range(len(result)):
    if (len(result[i][2])>3 and result[i][2][0] == 'HAN'):
        score.append(int(result[i][2][1])-int(result[i][2][2]))
    elif len(result[i][2])>3 and result[i][2][0] != 'HAN':
        score.append(int(result[i][2][2])-int(result[i][2][1]))
    #elif len(result[i][2])<3:
        #score.append('NULL')
print score
print len(score)

[-1, -3, 10, -7, 2, -1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4, -1, -4, 5, -2, 2, 1, 2, -2, 1, 5, 3, 5, -12, 2, 5, -3, 2, -2, -4, 3, -1, 1, -4, 4, -6, -3, 1, -4, -5, -7, 1, -6, -7, -1, 5, 1, -3, -1, -2, -4, -5, -5, 5, -8, -1, -1, 5, 6, -4, 3, 3, -2, -9, -4, 3, -1, -2, 4, 1, 1, -8, -1, -7, 2, 3, -1, 4, -3, -3, -8, 8, 2, -9, -2, -6, -6, -1, 0, -12, -14, 1, -6, -5, -4, 2, -1, 1, -6, -6, -4, -5, -6, 0, 6, -5, -3, 1, 5, -10, -6, 6, 8, -6, 2, -8, 1, -4, -9, -5, -5, -1, -4, -5, 1, -7, 0, 2, 3, -4, -4, -2, -2, -2, 3, -4, 2, -2, -3, -6, 1, -6, 4, -4, 1, -1, 4, -1, 3, -7, 2, 2, 5, 2, -12, 1, -8, -2, 1, 6, -3, -1, 1, 1, -3, 2, 2, -4, -1, 3, -1, 6, -15, 9, -1, 6, 3, -10, 1, -3, -14, -6, 4, 3, -8, 4, -4, 1, -3, -3, 2, -8, -1, 5, 1, -2, -2, -4, 5, -3, -6, -8, -8, 3, 7, -2, 2, -4, 6, -3, 2, -9, 4, 2, 1, -6, 0, -3, -4, -7, 1, 1, 3, 6, -3, 3, -6, 2, -1, -1, 11, -5, 2, 6, -8, 7, -1, -5, 1, 2, -3, 2, 2, -18, -3, -2, -3, -5, -6, 6, -1, -1, -5, 1, -5,

In [9]:
## train데이터, label데이터 생성 ##

train = []
label = []

for i in range(len(score)):
    if i+36>len(score):
            continue
    train.append([])
    for j in range(36):  
        train[i].append(score[i+j])
        
i=1
while(1):
    if i==len(train):
        break
    label.append([])
    if train[i][0]<0:
        label[i-1].append(0)
        label[i-1].append(0)
        label[i-1].append(1)
    elif train[i][0]>0:
        label[i-1].append(1)
        label[i-1].append(0)
        label[i-1].append(0)
    else:
        label[i-1].append(0)
        label[i-1].append(1)
        label[i-1].append(0)
    i = i+1
        

print len(train)
print len(label)

906
905


In [7]:
for i in range(len(train)):
    print train[i]

[-1, -3, 10, -7, 2, -1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7]
[-3, 10, -7, 2, -1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4]
[10, -7, 2, -1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4, -1]
[-7, 2, -1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4, -1, -4]
[2, -1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4, -1, -4, 5]
[-1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4, -1, -4, 5, -2]
[-1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4, -1, -4, 5, -2, 2]
[-6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4, -1, -4, 5, -2, 2, 1]
[-1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3,

In [36]:
for i in range(len(label)):
    print label[i]

[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[1, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]


In [37]:
print train[1]
print train[2]
print train[3]
print label[1]
print label[2]

[-3, 10, -7, 2, -1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4]
[10, -7, 2, -1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4, -1]
[-7, 2, -1, -1, -6, -1, 1, 7, -1, 1, -5, -7, 1, 6, -15, -13, -5, 5, 5, -11, -3, -9, -2, -6, -3, -2, -7, -4, -1, -4]
[1, 0, 0]
[0, 0, 1]


In [38]:
nptrain = np.array(train)
nplabel = np.array(label)

In [39]:
print type(nptrain)

<type 'numpy.ndarray'>


In [40]:
f1 = open('KBOtest1_train.txt', 'wb')
f2 = open('KBOtest1_label.txt', 'wb')
import pickle
pickle.dump(nptrain, f1)
pickle.dump(nplabel, f2)
f1.close()
f2.close()

In [42]:
f = open('KBOtest1_train.txt', 'rb')
a = pickle.load(f)
print type(a)
print a

<type 'numpy.ndarray'>
[[ -1  -3  10 ...,  -3  -2  -7]
 [ -3  10  -7 ...,  -2  -7  -4]
 [ 10  -7   2 ...,  -7  -4  -1]
 ..., 
 [-12   3   8 ...,   6  -3   8]
 [  3   8   8 ...,  -3   8   5]
 [  8   8  -7 ...,   8   5   1]]
